코드 완성했고 90일중앙&평균 댓글수, 조회수 구하는 중입니다!

In [ ]:
pip install selenium webdriver-manager beautifulsoup4 pandas openpyxl

In [ ]:
import os
import time
import random
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urlparse

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

print("✅ 모듈 임포트 완료")


✅ 모듈 임포트 완료


In [ ]:
# 로컬 경로 설정 (본인 환경에 맞게 수정)
base_path = "C:/exolyt_project"
handle_path = os.path.join(base_path, "소셜핸들.xlsx")
result_csv_path = os.path.join(base_path, "exolyt_result.csv")
cookie_path = os.path.join(base_path, "exolyt_cookies.json")  # 너가 저장한 JSON 파일명으로

# 엑셀에서 Username 추출
handle_df = pd.read_excel(handle_path)
usernames = handle_df.iloc[:, 0].dropna().apply(lambda x: x.lstrip('@')).tolist()

print("✅ 사용자 수:", len(usernames))
print("🎯 미리보기:", usernames[:])


✅ 사용자 수: 305
🎯 미리보기: ['theonlyhaven', 'turquoise.dec', 'ttalgi.ellie', 'srsaskin_', '_anjello', 'jf.skin24', 'cinthia_dosamples', 'lee.ly1012', 'prettywrinkles', 'xevelynluu', 'chloelouiseliveslife', 'melissahemmingsley', 'kbootyjenny', 'skincare.ninja', 'ugc_withjessi', 'so.calmama', 'jessicaroseugc', 'kayskwa', 'jennasiee', 'ugcxmichelle', 'autumnsaro', 'tinaangelinaaa', 'crystalschob3', 'life.w.lex4', 'jensnomz', 'srsdigitaldiary', 'eloise_ugc', 'shellllls_lifeeee', 'xokena_', 'mindfulugc', 'nooni.ugc', 'heyitsaliquinn', 'glo.ssary', 'ugc.karla', 'pretty.squirrelss', 'danielaapryl', 'allbriwoo', 'grw.alii_', 'jilliangottlieb', '_taniagrover', 'zhala.beauty', 'skinbykristin', 'lifewtamia', 'sunscreen.addict', 'toninlanaa', 'ft.angela', 'pinaydora', 'yangmediary', 'd_dudley4', 'beauty__bymegs', 'witandarte', 'beautycloudlove', 'psdiwan22', 'auraofmahmuda', 'vickyoffishal', '__allaboutskincare__', 'klymi.na', 'jeracoolio', 'razan.alani', 'angelinajarboe', 'waves_for_dayzz', 'jmeans77',

In [ ]:
import json

def apply_login_cookies(driver, cookie_path):
    with open(cookie_path, 'r', encoding='utf-8') as f:
        json_data = json.load(f)

    cookies_raw = json_data.get('cookies', [])

    # 문자열인 쿠키 → 객체로 변환
    cookies = []
    for c in cookies_raw:
        if isinstance(c, str):
            try:
                cookies.append(json.loads(c))
            except:
                continue
        else:
            cookies.append(c)

    # 첫 방문 (쿠키 설정을 위해)
    driver.get("https://www.exolyt.com")
    time.sleep(2)

    for cookie in cookies:
        try:
            cookie.pop('domain', None)  # 도메인 충돌 방지
            driver.add_cookie(cookie)
        except Exception as e:
            print(f"⚠️ 쿠키 추가 실패: {cookie.get('name', 'N/A')} → {e}")


In [ ]:
def handle_track_account_popup(driver):
    try:
        # 1️⃣ 첫 번째 팝업: "Track account" 버튼 클릭
        track_button = WebDriverWait(driver, 3).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="__next"]/main/div[2]/div[2]/div/div/div/button'))
        )
        track_button.click()
        print("✅ 1단계: 'Track account' 클릭")

        # 2️⃣ 두 번째 팝업: "Great!" 버튼 클릭 (뜨면)
        try:
            great_button = WebDriverWait(driver, 3).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="__next"]/main/div[4]/div/div/div/button'))
            )
            great_button.click()
            print("✅ 2단계: 'Great!' 팝업 닫기 완료")
        except:
            print("ℹ️ 'Great!' 팝업 없음")

        # 3️⃣ 다시 한번 'Track account' 클릭 (Analytics 탭 열리게 하기 위해)
        time.sleep(1)
        try:
            track_button = WebDriverWait(driver, 3).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="__next"]/main/div[2]/div[2]/div/div/div/button'))
            )
            track_button.click()
            print("✅ 3단계: 'Track account' 두 번째 클릭 완료")
        except:
            print("ℹ️ 3단계에서 Track 버튼 없음 (이미 닫힘 가능성)")
    except:
        print("ℹ️ 'Account data is outdated' 팝업 없음")


In [ ]:
def crawl_exolyt(username):
    try:
        options = webdriver.ChromeOptions()
        # options.add_argument("--headless")  # 필요 시 주석 해제
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("start-maximized")
        options.add_argument("user-agent=Mozilla/5.0")

        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

        # ✅ 1. 로그인 쿠키 적용
        cookie_path = os.path.join(base_path, "exolyt_login_state.json")
        driver.get("https://www.exolyt.com")
        time.sleep(2)
        apply_login_cookies(driver, cookie_path)

        # ✅ 2. 유저 페이지 이동
        driver.get(f"https://exolyt.com/user/tiktok/{username}")
        time.sleep(random.uniform(4, 6))
        handle_track_account_popup(driver)

        # ✅ 3. 로딩 완료 대기
        WebDriverWait(driver, 10).until(
            EC.invisibility_of_element_located((By.CLASS_NAME, "ds-fade-in"))
        )

        # ✅ 4. Overview 클릭
        overview_btn = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Overview')]"))
        )
        driver.execute_script("arguments[0].click();", overview_btn)
        print("✅ Overview 클릭 완료")
        time.sleep(2)

        "arguments[0].click();", checkbox_btn)
        print("✅ 'Choose date range' 체크박스 클릭 완료")
        time.sleep(2)# ✅ 5. 'Choose date range' 체크박스 클릭
        checkbox_btn = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//input[@type='checkbox' and contains(@class, 'cursor-pointer')]"))
        )
        driver.execute_script(

        # ✅ 최근 90일 필터 활성화
        label_btn = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//label[contains(text(), 'Choose date range')]"))
        )
        driver.execute_script("arguments[0].click();", label_btn)
        time.sleep(1.5)

        last_30_btn = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Last 30 days')]"))
        )
        driver.execute_script("arguments[0].click();", last_30_btn)
        time.sleep(1.5)

        last_90_btn = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//span[contains(text(), 'Last 90 days')]"))
        )
        driver.execute_script("arguments[0].click();", last_90_btn)
        time.sleep(2)


        # ✅ 'Choose date range' 체크박스 클릭 (이미 체크되었으면 스킵)
        checkbox_btn = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//input[@type='checkbox' and contains(@class, 'cursor-pointer')]"))
        )
        is_checked = driver.execute_script("return arguments[0].checked;", checkbox_btn)
        if not is_checked:
            driver.execute_script("arguments[0].click();", checkbox_btn)
            print("✅ 'Choose date range' 체크박스 클릭 완료")
        else:
            print("ℹ️ 'Choose date range' 체크박스는 이미 선택되어 있음")
        time.sleep(2)

        # ✅ 필터 적용 후 HTML이 업데이트될 때까지 대기
        time.sleep(4)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//div[contains(text(), 'Median views')]"))
        )


        # ✅ 6. HTML 파싱
        soup = BeautifulSoup(driver.page_source, "html.parser")

         # ✅ 7. 값 추출 함수들
        def extract_value(label):
            tag = soup.find("div", string=lambda t: t and label.lower() in t.lower())
            if tag:
                value_tag = tag.find_next_sibling("div")
                return value_tag.get_text(strip=True) if value_tag else None
            return None

        data = {
            "username": username,
            "median_views": extract_value("Median views"),
            "median_comments": extract_value("Median comments"),
            "avg_views": extract_value("Avg. Views"),
            "avg_comments": extract_value("Avg. Comments")
        }

        print("📊 최종 수집 데이터:", data)
        return data

    except Exception as e:
        print(f"❌ {username} - 실패: {e}")
        return None

    finally:
        try:
            driver.quit()
        except:
            pass


In [ ]:
results = []

for i, username in enumerate(usernames):
    print(f"\n[{i+1}/{len(usernames)}] ▶ {username}")
    result = crawl_exolyt(username)
    if result:
        results.append(result)
        print(f"✅ {username} - 완료")
    else:
        print(f"❌ {username} - 저장되지 않음")


[1/305] ▶ theonlyhaven
ℹ️ 'Account data is outdated' 팝업 없음
✅ Overview 클릭 완료
✅ 'Choose date range' 체크박스 클릭 완료
ℹ️ 'Choose date range' 체크박스는 이미 선택되어 있음
📊 최종 수집 데이터: {'username': 'theonlyhaven', 'median_views': '14.7K', 'median_comments': '6', 'avg_views': '31.6K', 'avg_comments': '13'}
✅ theonlyhaven - 완료

[2/305] ▶ turquoise.dec
ℹ️ 'Account data is outdated' 팝업 없음
✅ Overview 클릭 완료
✅ 'Choose date range' 체크박스 클릭 완료
ℹ️ 'Choose date range' 체크박스는 이미 선택되어 있음
📊 최종 수집 데이터: {'username': 'turquoise.dec', 'median_views': '40.3K', 'median_comments': '16', 'avg_views': '65.4K', 'avg_comments': '22'}
✅ turquoise.dec - 완료

[3/305] ▶ ttalgi.ellie
ℹ️ 'Account data is outdated' 팝업 없음
✅ Overview 클릭 완료
✅ 'Choose date range' 체크박스 클릭 완료
ℹ️ 'Choose date range' 체크박스는 이미 선택되어 있음
📊 최종 수집 데이터: {'username': 'ttalgi.ellie', 'median_views': '858', 'median_comments': '3', 'avg_views': '9.9K', 'avg_comments': '3'}
✅ ttalgi.ellie - 완료

[4/305] ▶ srsaskin_
ℹ️ 'Account data is outdated' 팝업 없음
✅ Overview 클릭 완료
✅ 'Choose d

In [ ]:
result_df = pd.DataFrame(results)
result_df.to_csv(result_csv_path, index=False, encoding='utf-8-sig')
print(f"\n📁 저장 완료: {result_csv_path}")



📁 저장 완료: C:/Users/dndma/Desktop/exolyt_project/exolyt_result.csv


In [ ]:
# 전체 usernames: 305개
print("총 username 수:", len(usernames))

# 저장된 결과: result_df 또는 results
# result_df 사용 시:
crawled_usernames = set(result_df['username'])

# 누락된 username 찾기
missed_usernames = [u for u in usernames if u not in crawled_usernames]

print(f"❌ 누락된 사용자 수: {len(missed_usernames)}")
print("🔍 누락된 사용자 미리보기:", missed_usernames[:10])


총 username 수: 305
❌ 누락된 사용자 수: 181
🔍 누락된 사용자 미리보기: ['_anjello', 'jf.skin24', 'prettywrinkles', 'kbootyjenny', 'so.calmama', 'jennasiee', 'ugcxmichelle', 'crystalschob3', 'srsdigitaldiary', 'eloise_ugc']


In [ ]:
missed_df = pd.DataFrame(missed_usernames, columns=['username'])
missed_path = os.path.join(base_path, "exolyt_missed_usernames.csv")
missed_df.to_csv(missed_path, index=False, encoding='utf-8-sig')
print(f"📁 누락 사용자 CSV 저장 완료: {missed_path}")


PermissionError: [Errno 13] Permission denied: 'C:/Users/dndma/Desktop/exolyt_project/exolyt_missed_usernames.csv'

In [ ]:
missed_df = pd.read_csv(os.path.join(base_path, "exolyt_missed_usernames.csv"))
missed_usernames = missed_df['username'].dropna().tolist()

results_retry = []

for i, username in enumerate(missed_usernames):
    print(f"\n[재시도 {i+1}/{len(missed_usernames)}] ▶ {username}")
    result = crawl_exolyt(username)
    if result:
        results_retry.append(result)
        print(f"✅ {username} - 재수집 완료")
    else:
        print(f"❌ {username} - 실패")

# 결과 병합 및 저장
results_total = result_df.to_dict('records') + results_retry
df_total = pd.DataFrame(results_total)
df_total.to_csv(os.path.join(base_path, "exolyt_result_retry_포함.csv"), index=False, encoding='utf-8-sig')
print("📁 전체 결과 저장 완료")



[재시도 1/181] ▶ _anjello
ℹ️ 'Account data is outdated' 팝업 없음
✅ _anjello - 재수집 완료

[재시도 2/181] ▶ jf.skin24
✅ 1단계: 'Track account' 클릭
ℹ️ 'Great!' 팝업 없음
ℹ️ 3단계에서 Track 버튼 없음 (이미 닫힘 가능성)
❌ jf.skin24 - 실패: Message: 

❌ jf.skin24 - 실패

[재시도 3/181] ▶ prettywrinkles
ℹ️ 'Account data is outdated' 팝업 없음
✅ prettywrinkles - 재수집 완료

[재시도 4/181] ▶ kbootyjenny
ℹ️ 'Account data is outdated' 팝업 없음
❌ kbootyjenny - 실패: Message: 
Stacktrace:
	GetHandleVerifier [0x0034FC03+61635]
	GetHandleVerifier [0x0034FC44+61700]
	(No symbol) [0x001705D3]
	(No symbol) [0x001B899E]
	(No symbol) [0x001B8D3B]
	(No symbol) [0x00200E12]
	(No symbol) [0x001DD2E4]
	(No symbol) [0x001FE61B]
	(No symbol) [0x001DD096]
	(No symbol) [0x001AC840]
	(No symbol) [0x001AD6A4]
	GetHandleVerifier [0x005D4523+2701795]
	GetHandleVerifier [0x005CFCA6+2683238]
	GetHandleVerifier [0x005EA9EE+2793134]
	GetHandleVerifier [0x003668C5+155013]
	GetHandleVerifier [0x0036CFAD+181357]
	GetHandleVerifier [0x00357458+92440]
	GetHandleVerifier [0x0035760